# Footprint area calculation
The original model is developed by Dr. Ayman Nassar, Utah State University<br>
This model has done several small modifications based on Ayman's program in order to consistent with Rui's project.<br>
Create date: October 13th, 2021<br>
Latest update date: October 13th, 2021<br>
Main contact: Rui.Gao@usu.edu<br>


In [ ]:
import pandas as pd
import numpy as np
import calc_footprint_FFP_climatology as myfootprint_s
from openpyxl import Workbook
from osgeo import gdal
import rasterio
import cv2
from affine import Affine
import pyproj as proj
import scipy
import warnings
warnings.filterwarnings('ignore')

In [19]:
# A folder where you want to save the results
path_outfolder = r'D:'
# The file which support the footprint area calculation
df=pd.read_excel(r'D:\...\.xlsx',sheet_name='Sheet1')
file_num = len(df)

# Global constants
karman=0.4 #von Karman's constant
gravity=9.8 #acceleration of gravity (m s-2)
h_s = 3000 #Height of boundary layer [m]
dx =round(3.60,1) #spatial resolution for the output footprint 
origin_d = 500 #Model distance from origin [m]
Cp=1005.0                                               

df.head(3)

,No,Site,Year,Month,Day,Time_flight,Time_EC,Longitude,Latitude,DOY,...,Air_TempK,Incident_Solar,Net_Radiation,Soil_Heat_Flux,Sensible_Heat_Flux,Latent_Heat_Flux,Soil_Temp,Soil_Moisture_Content,Canopy_Height,Obukov
0,1,720-1,2018,6,18,06:34:00,06:30:00,-120.176064,36.849239,169,...,289.56,279.38,141.49,8.36,20.26,77.53,19.81,11.29,1.7583,-124.795394
1,2,720-1,2018,6,19,06:34:00,06:30:00,-120.176064,36.849239,170,...,290.89,283.97,148.43,4.06,10.36,66.28,21.08,11.16,1.7629,-56.062075
2,3,720-1,2018,6,19,11:20:00,11:30:00,-120.176064,36.849239,170,...,302.17,1008.00,712.49,92.15,110.20,407.10,26.58,11.60,1.7629,-18.163922


In [20]:
def mask_fp_cutoff(f_array,cutoff=0.9):
    '''
    Returns a np.ndarray that cuts off values below the cumulative .9 threshold
    with nans set to zero
    
    Inputs:
        f_array : 2-D array of point footprint weights [unitless]
        
    Outputs:
        2-D np.ndarray
    '''
    val_array = f_array.flatten()
    sort_df = pd.DataFrame({'f':val_array}).sort_values(by='f').iloc[::-1]
    sort_df['cumsum_f'] = sort_df['f'].cumsum()
    
    sort_group = sort_df.groupby('f',as_index=True).mean()
    diff = abs(sort_group['cumsum_f']-cutoff)
    sum_cutoff = diff.idxmin()
    f_array = np.where(f_array>=sum_cutoff,f_array,np.nan)
    f_array[~np.isfinite(f_array)] = 0.00000000e+000
    
    return f_array

In [21]:
def find_transform(xs,ys):
    '''
    Returns the affine transform for 2d arrays xs and ys
    
    Inputs:
        xs : 2-d grid of x coordinates (usually utm)
        ys : 2-d grid of y coordinates (usually utm)
        
    Outputs:
        aff_transform : affine.Affine object
    '''
    shape = xs.shape

    #Choose points to calculate affine transform
    y_points = [0, 0, shape[0] - 1]
    x_points = [0, shape[0] - 1, shape[1] - 1]
    in_xy = np.float32([[i, j] for i, j in zip(x_points, y_points)])
    out_xy = np.float32([[xs[i, j], ys[i, j]] for i, j in zip(y_points, x_points)])
    

    #Calculate affine transform
    aff_transform = Affine(*cv2.getAffineTransform(in_xy,out_xy).flatten())

    return aff_transform

In [22]:
## Calculates the latent heat of vaporization
def CalcLambda(Ta_K):
    '''Calculates the latent heat of vaporization
    
    Parameter
    ---------
    Ta_K : Air temperature (Kelvin)
    
    Results
    -------
    Lambda : Latent heat of vaporisation (MJ kg-1)
    
    based on Eq. 3-1 Allen FAO98 '''
    
    Lambda = 2.501 - (2.361e-3* (Ta_K-273.15) ) 
    
    return Lambda

In [23]:
## calculate the Monin-Obukhov lenght
def CalcL (ustar, Ta_K, rho, c_p, H, LE):
    '''Calculates the Monin-Obukhov lenght

    Parameters
    ----------
    ustar : friction velocity (m s-1)
    Ta_K : air temperature (Kelvin)
    rho : air density (kg m-3)
    c_p : Heat capacity of air at constant pressure (J kg-1 K-1)
    H : sensible heat flux (W m-2)
    LE : latent heat flux (W m-2)
    
    Returns
    -------
    L : Obukhov stability length (m)

    based on equation (2.46) from Brutsaert (2005): 
    Hydrology - An Introduction (pp 46)'''

    # first convert latent heat into rate of surface evaporation (kg m-2 s-1)
    Lambda = CalcLambda(Ta_K)*1e6 #in J kg-1
    E = LE / Lambda
    # Virtual sensible heat flux
    Hv=H+(0.61*Ta_K*c_p*E)
    if Hv!=0:
        L_const = karman*gravity/Ta_K
        L = -ustar**3 / ( L_const*(Hv/(rho*c_p) ))
    else:
        L = float('inf')
    return L

In [24]:
## estimate sigma_v using bounary layer height = 3000m
def CalcSigma_v(ustar,L):
    '''% estimate sigma_v using bounary layer height = 3000m
    
    Parameters
    ----------
    L : Obukhov length (m)
    ustar : friction velocity (m s-1)
    
    Returns
    -------
    sigma_v : standard deviation of the cross wind velocity'''
    
    sigma_v= ustar*(12.0 - 0.5*3000.0/L)**(1.0/3.0) 
    return sigma_v

In [40]:
for i in range(0,file_num):
    temp_df = df.loc[i,:]
    zm_s = temp_df['Height_Tower'] #Measurement height [m]
    h_c = temp_df['Canopy_Height'] #Height of canopy [m]
    Ta = temp_df['Air_TempC'] #Air temperature [C]

    # Calculate MO lenght
    rho=1.3079-0.0045*Ta    
    #L=CalcL (temp_df['ustar'], Ta+273.15, rho, Cp, temp_df['H'], temp_df['LE'])
    L=temp_df['Obukov']
    ## calculate sigmav
    sigmav=CalcSigma_v(temp_df['ustar'],L)

    #Convert local lat/lon to local UTM (Lodi: UTM 10).
    x_coor=round(temp_df['Longitude'],6)
    y_coor=round(temp_df['Latitude'],6)
    station_coord = (x_coor,y_coor) #Lodi lat/lon in WGS-84
    in_proj = proj.Proj(init='EPSG:4326')
    out_proj = proj.Proj(init='EPSG:32610')
    (station_x,station_y) = proj.transform(in_proj,out_proj,*station_coord)

    new_dat = None
    try:
        temp_line = temp_df
        #If you're debugging, verbosity=2 will give you error output from the model
        temp_ffp = myfootprint_s.FFP_climatology(domain=[-origin_d,origin_d,-origin_d,origin_d],dx=dx,dy=dx,
                                zm=zm_s,z0=h_c*0.15,h=h_s,rs=None,
                                ol=L,sigmav=sigmav, umean=temp_df['Wind_Speed'],
                                ustar=temp_df['ustar'], wind_dir=temp_df['Wind_Direction'],
                                crop=0,fig=0,verbosity=0) 
        f_2d = np.array(temp_ffp['fclim_2d'])
        x_2d = np.array(temp_ffp['x_2d']) + station_x
        y_2d = np.array(temp_ffp['y_2d']) + station_y
        f_2d = f_2d*dx**2

        #Calculate affine transform for given x_2d and y_2d
        affine_transform = find_transform(x_2d,y_2d)

        if new_dat is None:
            #First input to the function is the output file name
            #Change the crs to whatever utm you are in as a proj4 string
            filename = str(temp_df['Site'])+"_"+str(temp_df['Year'])+str(temp_df['Month'])+str(temp_df['Day'])+str(temp_df['Time_flight'].hour)+str(temp_df['Time_flight'].minute)+".tif"
            print("Saved filename is:",filename)
            folder_filename = path_outfolder+'\\'+filename
            new_dat = rasterio.open(folder_filename,'w',driver='GTiff',dtype=rasterio.float64,
                                    count=1,height=f_2d.shape[0],width=f_2d.shape[1],
                                    transform=affine_transform,
                                    crs='+proj=utm +zone=10 +ellps=WGS84 +datum=WGS84 +units=m +no_defs',
                                    nodata=0.00000000e+000)
        #Mask out points that are below a % threshold (defaults to 90%)
        f_2d = mask_fp_cutoff(f_2d)
        #Write the new band
        new_dat.write(f_2d,1)
        new_dat.close()
        new_dat = None

    except Exception as e:
        print(e)
        print(f'Error')
        temp_ffp = None
        pass

720-1_2018618634.tif
720-1_2018619634.tif
720-1_20186191120.tif
720-1_20186191317.tif
720-1_20186191538.tif
720-1_2018712633.tif
720-1_20187121229.tif
720-1_20187121532.tif
720-1_2018713614.tif
720-1_20187131040.tif
720-1_20187131522.tif
720-1_201885633.tif
720-1_2018851044.tif
720-1_2018851233.tif
720-1_201886639.tif
720-1_2018861041.tif
720-1_2018861141.tif
720-1_201954622.tif
720-1_2019541025.tif
720-1_201954128.tif
720-2_201954622.tif
720-2_2019541025.tif
720-2_201954128.tif
720-3_201954622.tif
720-3_2019541025.tif
720-3_201954128.tif
720-4_201954622.tif
720-4_2019541025.tif
720-4_201954128.tif
720-4_2018618634.tif
720-4_2018619634.tif
720-4_20186191120.tif
720-4_20186191317.tif
720-4_20186191538.tif

Error
720-4_20187121229.tif
720-4_20187121532.tif
720-4_2018713614.tif
720-4_20187131040.tif
720-4_20187131522.tif
720-4_201885633.tif
720-4_2018851044.tif
720-4_2018851233.tif
720-4_201886639.tif
720-4_2018861041.tif
720-4_2018861141.tif
720-3_2018618634.tif
720-3_2018619634.tif
720-